In [64]:
import os
import re
from tqdm import tqdm
import os
import sys
from PIL import Image
import yaml
from pathlib import Path
import torchvision
import torchvision.transforms.functional as TF
import torch

root_dir = os.path.abspath(os.path.join(os.getcwd(), ".."))

if root_dir not in sys.path:
    sys.path.append(root_dir)

from utils.dir_utils import mkdir, get_last_path
from utils.model_utils import load_checkpoint
from model.URSCT_model import URSCT



In [9]:
def is_image_file(filename):
    return any(filename.endswith(extension) for extension in ['.png', '.jpg', '.jpeg'])

def get_all_images_in_directory(directory):
    images = []
    for root, dirs, files in os.walk(directory):
        for file in files:
            if is_image_file(file):
                images.append(os.path.join(root, file))
    return images
# add all images in the directory to the list

original_dataset_path = '../../eda/data/DeepFish/'

yolo_images = get_all_images_in_directory('../../yolo/datasets/DeepFish-2')
original_images = get_all_images_in_directory(original_dataset_path)

len(yolo_images), len(original_images)

(6517, 47406)

In [14]:
yolo_images[:5]

['../../yolo/datasets/DeepFish-2/valid/images/7268_F1_f000626_jpg.rf.e67bc389f7de00abc8f3043b41c801b2.jpg',
 '../../yolo/datasets/DeepFish-2/valid/images/9866_no_fish_f000178_jpg.rf.b968f434894009e2cc783e7eb47e0a4b.jpg',
 '../../yolo/datasets/DeepFish-2/valid/images/7268_F1_f000433_jpg.rf.da0b0e1e158f9986f2cf27ada0f50b83.jpg',
 '../../yolo/datasets/DeepFish-2/valid/images/7490_F3_f000042_jpg.rf.8de37b4a1a44c5e9971cdcf523a836a7.jpg',
 '../../yolo/datasets/DeepFish-2/valid/images/9894_no_fish_2_f000055_jpg.rf.df2ade00ec3b946cd5cad48faa585ee1.jpg']

In [17]:
# goal:
# for each image with yolo annotation, get original image
# for each filtered image, run swin transformer
# write the output to a new directory with division test train val
# transform yolo annotations to the new image size (multiply y axis by 16/9)

habitat_frame_regex = re.compile(r'(\d{4})_(.+)_(f\d+)[_.](?=jpg|png)')
yolo_images_groups = [re.match(habitat_frame_regex, image.split('/')[-1]).groups(0) for image in yolo_images]

# create set for constant time lookup
yolo_images_set = set(yolo_images_groups)

list(yolo_images_set)[:4]

[('7490', 'NF2', 'f000054'),
 ('9894', 'no_fish_2', 'f000079'),
 ('7426', 'NF1', 'f000204'),
 ('7482', 'F1', 'f000045')]

In [43]:
original_images_dict = {
    re.match(habitat_frame_regex, image.split("/")[-1]).groups(0): image
    for image in original_images
}
# lowercase keys in the dictionary
original_images_dict = {
    tuple(
        key_part.lower() if isinstance(key_part, str) else key_part for key_part in key
    ): value
    for key, value in original_images_dict.items()
}


len(original_images_dict), len(original_images)  # duplicate keys are removed, i hope this is not bad

(39766, 47406)

In [44]:
list(original_images_dict.items())[:4]

[(('9892', 'acanthopagrus_palmaris', 'f000038'),
  '../../eda/data/DeepFish/Classification/9892/valid/9892_acanthopagrus_palmaris_f000038.jpg'),
 (('9892', 'acanthopagrus_palmaris', 'f000010'),
  '../../eda/data/DeepFish/Localization/masks/valid/9892_acanthopagrus_palmaris_f000010.png'),
 (('9892', 'acanthopagrus_palmaris', 'f000004'),
  '../../eda/data/DeepFish/Classification/9892/valid/9892_acanthopagrus_palmaris_f000004.jpg'),
 (('9892', 'acanthopagrus_palmaris_2', 'f000008'),
  '../../eda/data/DeepFish/Classification/9892/valid/9892_Acanthopagrus_palmaris_2_f000008.jpg')]

In [59]:
with open('../configs/Enh_opt.yaml', 'r') as config:
    opt = yaml.safe_load(config)
    opt_test = opt['DEEPFISH']

device = opt_test['DEVICE']
model_detail_opt = opt['MODEL_DETAIL']
result_dir = os.path.join(opt_test['SAVE_DIR'], opt['TRAINING']['MODEL_NAME'], 'test_results')
mkdir(result_dir)

model = URSCT(model_detail_opt).to(device)
path_chk_rest = get_last_path(os.path.join(opt_test['SAVE_DIR'], opt['TRAINING']['MODEL_NAME'], 'models'), '_bestSSIM.pth')
load_checkpoint(model, path_chk_rest, device)
model.eval()

patch_size = opt_test['TEST_PS']

Model loading successfully!


In [69]:
for yolo_image in tqdm(yolo_images):
    habitat, fish_type, frame = re.match(
        habitat_frame_regex, yolo_image.split("/")[-1]
    ).groups(0)
    original_image_path = original_images_dict.get((habitat, fish_type.lower(), frame))

    if original_image_path is None:
        raise ValueError(
            f"Original image not found for {yolo_image} with key {(habitat, fish_type.lower(), frame)}"
        )

    # generate swin image
    inp_img = Image.open(original_image_path).convert("RGB")
    inp_img = TF.to_tensor(inp_img)
    inp_img = TF.resize(inp_img, (patch_size[0], patch_size[1]))

    image_gpu = inp_img.to(device).unsqueeze(0)

    with torch.no_grad():
        restored_SR = model(image_gpu)

    # save swin image to output directory
    yolo_image_path = Path(yolo_image)
    model_stage = str(yolo_image_path.parent.parent)

    # create directories if they don't exist
    mkdir(os.path.join(result_dir, model_stage, 'images'))

    torchvision.utils.save_image(
        restored_SR[0], 
        os.path.join(result_dir, model_stage, 'images', yolo_image_path.stem + '.png')
    )

    #  TODO save yolo annotations to output directory

  2%|▏         | 98/6517 [01:01<1:06:55,  1.60it/s]


KeyboardInterrupt: 

In [12]:
# original_yolo_images = []
# spotted = {}

# for image in original_images:
#     habitat, frame = re.match(habitat_frame_regex, image.split("/")[-1]).groups(0)

#     if (habitat, frame) in yolo_images_set:
#         original_yolo_images.append(image)

#         if (habitat, frame) in spotted.keys():
#             print(
#                 f"Duplicate: {habitat} {frame}",
#                 spotted[(habitat, frame)],
#                 image,
#             )

#         spotted[(habitat, frame)] = image

# len(original_yolo_images)

Duplicate: 9892 f000024 ../../eda/data/DeepFish/Classification/9892/valid/9892_Acanthopagrus_palmaris_2_f000024.jpg ../../eda/data/DeepFish/Classification/9892/valid/9892_Caranx_f000024.jpg
Duplicate: 9892 f000030 ../../eda/data/DeepFish/Classification/9892/valid/9892_Acanthopagrus_palmaris_2_f000030.jpg ../../eda/data/DeepFish/Classification/9892/valid/9892_Caranx_f000030.jpg
Duplicate: 9892 f000018 ../../eda/data/DeepFish/Classification/9892/valid/9892_Acanthopagrus_palmaris_2_f000018.jpg ../../eda/data/DeepFish/Classification/9892/valid/9892_Caranx_f000018.jpg
Duplicate: 9892 f000042 ../../eda/data/DeepFish/Classification/9892/valid/9892_Caranx_f000042.jpg ../../eda/data/DeepFish/Classification/9892/valid/9892_Acanthopagrus_palmaris_2_f000042.jpg
Duplicate: 9892 f000019 ../../eda/data/DeepFish/Classification/9892/valid/9892_Acanthopagrus_palmaris_2_f000019.jpg ../../eda/data/DeepFish/Classification/9892/valid/9892_Caranx_f000019.jpg
Duplicate: 9892 f000031 ../../eda/data/DeepFish/Cl

19651